# BC

In [1]:
class LinkedList:
    class Node:
        def __init__(self, num, prev):
            self.num = num
            self.prev = prev
            self.next = None

    def __init__(self, num, start):
        self.root = self.Node(0, None)
        self.current = None
        self.stack = []
        temp = self.root
        for i in range(1, num):
            new_node = self.Node(i, temp)
            temp.next = new_node
            if i == start:
                self.current = new_node
            temp = new_node

    def up(self, num):
        for _ in range(num):
            if self.current.prev:
                self.current = self.current.prev

    def down(self, num):
        for _ in range(num):
            if self.current.next:
                self.current = self.current.next

    def remove(self):
        remove_node = self.current
        self.stack.append(remove_node)
        if remove_node.next:
            if remove_node == self.root:
                self.root = remove_node.next
            self.current = remove_node.next
            self.current.prev = remove_node.prev
            if remove_node.prev:
                remove_node.prev.next = self.current
        else:
            self.current = remove_node.prev
            self.current.next = None

    def recover(self):
        recover_node = self.stack.pop()
        if recover_node.prev:
            recover_node.prev.next = recover_node
        if recover_node.next:
            recover_node.next.prev = recover_node
            if recover_node.next == self.root:
                self.root = recover_node

    def get_root(self):
        return self.root

    def __bool__(self):
        return True


def solution(n, k, cmd):
    table = LinkedList(n, k)
    for c in cmd:
        if c[0] == 'U':
            table.up(int(c.split()[1]))
        elif c[0] == 'D':
            table.down(int(c.split()[1]))
        elif c[0] == 'C':
            table.remove()
        else:
            table.recover()
    node = table.get_root()
    result = ["X"] * n
    while node:
        result[node.num] = "O"
        node = node.next
    return "".join(result)


## 투 포인터(내 풀이), 효율성 2/10

In [3]:
from collections import deque


def solution(n, k, cmd):
    stack = []
    l = [i for i in range(n)]
    db = set(i for i in range(n))
    l_q, r_q = deque(l[:k]), deque(l[k:])

    for c in cmd:
        if not r_q:
            r_q.append(l_q.pop())

        if r_q and c == 'C':
            item = r_q.popleft()
            stack.append((item, len(l_q)))  # 복구를 위해 삭제될 때 위치를 저장해야함
            db.remove(item)

        elif c == 'Z':  # 복구할 때 인덱스 처리 -> 삭제 전 위치로 돌아감
            now = r_q[0]
            item, pos = stack.pop()
            temp = list(l_q + r_q)
            temp.insert(pos, item)
            now = temp.index(now)
            l_q, r_q = deque(temp[:now]), deque(temp[now:])
            db.add(item)
        else:
            c, idx = c.split()
            idx = int(idx)
            if c == 'U':
                for _ in range(idx):
                    if l_q:
                        r_q.appendleft(l_q.pop())
                    else:
                        break
            elif c == 'D':
                for _ in range(idx):
                    if r_q:
                        l_q.append(r_q.popleft())
                    else:
                        break

    answer = ['O' if i in db else 'X' for i in range(n)]
    return ''.join(answer)


## 계속 기록, 효율성 0/10

In [4]:
from collections import deque, defaultdict
from copy import deepcopy


def solution(n, k, cmd):
    stack = []
    table = defaultdict(list)
    now = 1
    table[0] = [i for i in range(n)]
    for i, c in enumerate(cmd):
        table[i+1] = deepcopy(table[i])
        if c == 'C':
            item = table[i+1].pop(k)
            stack.append((item, k))  # 복구를 위해 삭제될 때 위치를 저장해야함
            if k >= len(table[i+1]):
                k = len(table[i+1]) - 1

        elif c == 'Z':  # 복구할 때 인덱스 처리 -> 삭제 전 위치로 돌아감
            item, pos = stack.pop()
            table[i+1].insert(pos, item)
            if pos <= k:
                k = k + 1

        else:
            c, idx = c.split()
            idx = int(idx)
            if c == 'U':
                k = k - idx
            elif c == 'D':
                k = k + idx

    result = table[len(cmd)]
    answer = ['O' if i in result else 'X' for i in range(n)]
    return ''.join(answer)
